In [1]:
import math
import matplotlib.pyplot as plt
import numpy as np
from pycocotools.coco import COCO
import random
import tensorflow as tf
from tqdm import tqdm

TRANSFER = True

In [2]:
# https://cs230.stanford.edu/blog/datapipeline/

# Create a list of filenames and labels for the dataset
def create_pairs(location = '/home/gregory/Datasets/COCO/2017', mode = 'val'):
    
    # Each 'label' vector is large enough for easy indexing, but this means it contains unused indices
    dim = 91
    
    file = '{}/annotations/instances_{}2017.json'.format(location, mode)
    
    coco = COCO(file)
    
    images = coco.loadImgs(coco.getImgIds())
    
    filenames = []
    labels = []
    
    for i in range(len(images)):
    
        im_obj = images[i]
        
        filenames.append('{}/{}2017/{}'.format(location, mode, im_obj['file_name']))
        
        annotations = coco.loadAnns(coco.getAnnIds(im_obj['id'], iscrowd=None))
        label = np.zeros((dim), dtype = np.float32)
        for x in annotations:
            label[x['category_id']] = 1.0
        labels.append(label)
        
    filenames = np.array(filenames)
    labels = np.array(labels)
        
    return filenames, labels
    
def parse_function(filename, label):
    image_string = tf.io.read_file(filename)

    #Don't use tf.image.decode_image, or the output shape will be undefined
    image = tf.image.decode_jpeg(image_string, channels=3)

    #This will convert to float values in [0, 1]
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize_with_pad(image, 224, 224)
    
    return image, label

def create_dataset(filenames, labels, batch_size = 32, num_parallel_calls = 4):

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.shuffle(len(filenames))
    dataset = dataset.map(parse_function, num_parallel_calls=num_parallel_calls)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(1)

    return dataset

In [3]:
# Load a pre-trained model

model_base = tf.keras.applications.MobileNetV2(input_shape = (224, 224, 3),
                                               include_top=False,
                                               pooling = 'avg',
                                               weights='imagenet') #ResNet50
model_base.trainable = False

model = tf.keras.Sequential([
  model_base,
  tf.keras.layers.Dense(91)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 1280)              2257984   
_________________________________________________________________
dense (Dense)                (None, 91)                116571    
Total params: 2,374,555
Trainable params: 116,571
Non-trainable params: 2,257,984
_________________________________________________________________


In [4]:
model_location = './model_transfer/weights.h5'

if TRANSFER: 
    learning_rate = 0.001
    learning_rate_decay = 0.3
    learning_rate_drops = 3
    min_epochs = 2
    stopping_epochs = 2
    stopping_tol = 0.001
    batch_size = 32

    folder = '/home/gregory/Datasets/COCO/2017'

    f, l = create_pairs(mode = 'train')
    n_train = len(f)

    data_train = create_dataset(f, l, batch_size = batch_size)

    f, l = create_pairs(mode = 'val')
    n_val = len(f)

    data_val = create_dataset(f, l, batch_size = batch_size)

    n_batches_train = math.floor(n_train / batch_size)
    n_batches_val = math.floor(n_val / batch_size)

    def loss(model, inputs, targets):
        preds = model(inputs)
        return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = targets, logits = preds))

    def grad(model, inputs, targets):
        with tf.GradientTape() as tape:
            loss_value = loss(model, inputs, targets)
        return loss_value, tape.gradient(loss_value, model.trainable_variables)

    # Setup the initial optimizer (it will be re-initialized when the learning rate gets dropped)
    optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

    # Basic counters for the training process
    epoch = 0
    best_epoch = 0
    best_loss = np.inf
    drops = 0

    # Run the training loop
    while True:

        # Check the stopping condition
        if epoch - best_epoch > stopping_epochs and epoch > min_epochs:
            # We have finished only if we have fully decayed the learning_rate
            if drops == learning_rate_drops:
                break
            else:
                print("Dropping learning_rate")
                learning_rate *= learning_rate_decay
                optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)
                drops += 1

        # Train for an epoch
        epoch_loss_avg = tf.keras.metrics.Mean()
        for (x_batch, y_batch) in tqdm(data_train):
            loss_value, grads = grad(model, x_batch, y_batch)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            epoch_loss_avg.update_state(loss_value)
        epoch_loss = epoch_loss_avg.result().numpy()

        # Calculate the validation loss
        epoch_loss_avg_val = tf.keras.metrics.Mean()
        for (x_batch, y_batch) in data_val:
            loss_value = loss(model, x_batch, y_batch)
            epoch_loss_avg_val.update_state(loss_value)
        value = epoch_loss_avg_val.result().numpy()

        # Check if we have made progress
        if value < best_loss - stopping_tol:
            print("Epoch / Epoch Train Loss / Val Loss: " + str(epoch) + " " + str(epoch_loss) + " " + str(value) + " -> saving")
            best_loss = value
            best_epoch = epoch
            model.save_weights(model_location)
        else:
            print("Epoch / Epoch Train Loss / Val Loss: " + str(epoch) + " " + str(epoch_loss) + " " + str(value))

        # Update counters
        epoch += 1
        
model.load_weights(model_location)


loading annotations into memory...
Done (t=9.05s)
creating index...
index created!
loading annotations into memory...


0it [00:00, ?it/s]

Done (t=0.37s)
creating index...
index created!


3697it [09:56,  6.20it/s]
0it [00:00, ?it/s]

Epoch / Epoch Train Loss / Val Loss: 0 0.07064531 0.06640391 -> saving


...

Epoch / Epoch Train Loss / Val Loss: 5 0.05771087 0.06376489
Dropping learning_rate


3697it [09:23,  6.57it/s]
0it [00:00, ?it/s]

Epoch / Epoch Train Loss / Val Loss: 6 0.056496873 0.06330037
Dropping learning_rate


3697it [09:25,  6.54it/s]


Epoch / Epoch Train Loss / Val Loss: 7 0.056047935 0.063010454


In [6]:
batch_size = 32

f, l = create_pairs(mode = 'val')
n_val = len(f)

data_val = create_dataset(f, l, batch_size = batch_size)

n_batches_val = math.ceil(n_val / batch_size)

counts = []

for (x_batch, y_batch) in data_val:
    
    y_hat_batch = 1.0 * (tf.math.sigmoid(model(x_batch)).numpy() >= 0.5)
    
    counts_batch = np.sum(1.0 * (y_hat_batch == y_batch.numpy()), axis = 0)
    
    counts.append(counts_batch)
    
acc = np.sum(np.array(counts), axis = 0) / n_val
    
print("Per Object Validation Accuracy: ", acc)
print("Average Validation Accuracy", np.mean(acc))
        

loading annotations into memory...
Done (t=0.36s)
creating index...
index created!
Per Object Validation Accuracy:  [1.     0.863  0.9746 0.9132 0.9816 0.9922 0.9782 0.988  0.9548 0.9862
 0.9728 0.988  1.     0.9912 0.994  0.9578 0.9832 0.9832 0.9764 0.9834
 0.9908 0.9876 0.9944 0.9962 0.9968 0.9952 1.     0.955  0.9754 1.
 1.     0.9438 0.9808 0.981  0.9874 0.9894 0.9892 0.9738 0.9908 0.9896
 0.9908 0.9856 0.9892 0.9898 0.9298 1.     0.9788 0.9248 0.9726 0.9624
 0.9686 0.9496 0.9832 0.9852 0.9816 0.9878 0.9924 0.9866 0.9924 0.9842
 0.9898 0.9806 0.9012 0.9678 0.9672 0.9796 1.     0.9242 1.     1.
 0.9852 1.     0.97   0.9744 0.9896 0.9736 0.9864 0.961  0.9892 0.983
 0.9984 0.98   0.983  1.     0.9582 0.9748 0.9776 0.9954 0.991  0.9982
 0.9932]
Average Validation Accuracy 0.978978021978022
